In [1]:
from uuid import uuid4
import os

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StdOutCallbackHandler
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from qdrant_client.models import PointStruct
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import create_retrieval_chain
from langchain_qdrant import Qdrant

In [2]:
MODEL = "mistral"
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

C:\Users\mayur\AppData\Local\Temp\ipykernel_2744\1845413169.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model=MODEL)
C:\Users\mayur\AppData\Local\Temp\ipykernel_2744\1845413169.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=MODEL)


In [ ]:
client.delete_collection(collection_name= "ros2")

In [3]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="demo_collection",
    vectors_config=VectorParams(size=3072, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection",
    embedding=embeddings,
)

QdrantVectorStoreError: Existing Qdrant collection is configured for dense vectors with 3072 dimensions. Selected embeddings are 4096-dimensional. If you want to recreate the collection, set `force_recreate` parameter to `True`.

In [4]:
##https://arxiv.org/pdf/2404.18208
loader = PyPDFLoader("./knowledge/ML.pdf")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
document = loader.load_and_split(text_splitter)

In [5]:
document[0]


Document(metadata={'source': './knowledge/ML.pdf', 'page': 0}, page_content='1. Standardization \n2. Normalization \n3. Evaluation matrix \n4. Cross validation \n5. Variance and covariance \n6. Anomaly detection \n7. Outliers \n8. Mathematics for data science \n9. Numpy \n10. Pandas')

In [6]:
len(document)


44

In [9]:
from qdrant_client import QdrantClient
client = QdrantClient(":memory:")

In [10]:
client.create_collection(
    collection_name="ml",
    vectors_config={
        "content": VectorParams(size=5120, distance=Distance.COSINE)
        }
    )

True

In [11]:
def chunked_metadata(data, client=client, collection_name="ml"):

    chunked_metadata = []

    for item in data:

      id = str(uuid4())
      content = item.page_content
      source = item.metadata["source"]
      page = item.metadata["page"]

      content_vector = embeddings.embed_documents([content])[0]
      vector_dict = {"content": content_vector}

      payload = {
           "page_content": content,
           "metadata": {
                        "id": id,
                        "page_content": content,
                        "source": source,
                        "page": page,
                        }
            }


      metadata = PointStruct(id=id, vector=vector_dict, payload=payload)
      chunked_metadata.append(metadata)

    client.upsert(
                  collection_name=collection_name,
                  wait=True,
                  points=chunked_metadata)

In [16]:
chunked_metadata(document[:5])


ValueError: could not broadcast input array from shape (4096,) into shape (5120,)

In [13]:
client.get_collections()


CollectionsResponse(collections=[CollectionDescription(name='ml')])

In [14]:
ros_collection = client.get_collection("ml")

print(f"Points: {ros_collection.points_count} ")

Points: 1 


In [15]:
client.search(
    collection_name="ml",
    query_vector=("content", embeddings.embed_documents(["FPGA Platforms"])[0]),
    with_payload=["page_content", "source"],
    limit=5
)

ValueError: shapes (0,5120) and (4096,) not aligned: 5120 (dim 1) != 4096 (dim 0)

In [14]:
vectorstore = Qdrant(client=client,
                     collection_name="ros2",
                     embeddings=embeddings,
                     vector_name="content")

In [15]:
template = """

You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \


Question: {input}
Context: {context}

Answer:

"""

retriever = vectorstore.as_retriever()

In [16]:
prompt = ChatPromptTemplate.from_template(template)
handler =  StdOutCallbackHandler()


In [17]:
combine_docs_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [18]:
result = retrieval_chain.invoke({"input": "how to use FPGA"})


In [19]:
result

{'input': 'how to use FPGA',
 'context': [Document(page_content='thousands-fold faster communications and better energy-efficiency, growing closer to the realm of hu-\nman brain-like speeds and efficiency.\nIn the fascinating realm of neuroscience, a single neuron firing an action potential in the human\nbrain lasts approximately 2 to 5 milliseconds, consuming an incredibly minute amount of energy —\nroughly 0.03 to 0.3 microjoules. These latencies and energy-efficiency set a benchmark that current', metadata={'id': '6ea093db-3bdb-453f-9ee9-a7bf37244d33', 'page': 0, 'page_content': 'thousands-fold faster communications and better energy-efficiency, growing closer to the realm of hu-\nman brain-like speeds and efficiency.\nIn the fascinating realm of neuroscience, a single neuron firing an action potential in the human\nbrain lasts approximately 2 to 5 milliseconds, consuming an incredibly minute amount of energy —\nroughly 0.03 to 0.3 microjoules. These latencies and energy-efficiency 

In [20]:
result["answer"]

' To use an FPGA (Field Programmable Gate Array) in the context of achieving brain-like speeds and efficiency in robotic networking as discussed in ROS 2 on aChip, you would follow these general steps:\n\n1. Identify your requirements for real-time communication in the robotic system that you aim to improve with an FPGA solution.\n\n2. Select an appropriate FPGA device based on factors like processing power, energy efficiency, and compatibility with existing hardware and software components of your project.\n\n3. Acquire a development board or chip for experimentation and implementation purposes; popular choices include Xilinx and Intel (formerly Altera) boards.\n\n4. Familiarize yourself with the FPGA programming environment and tools, such as Vivado Design Suite by Xilinx or Quartus Prime by Intel. These software suites help you in designing your custom logic circuits on the FPGA.\n\n5. Implement ROS 2 protocols for pubsub (publish-subscribe) communications using a high-level hardwar